In [1]:
# import random as rnd
# import secrets as scr

from datetime import datetime
from random import SystemRandom

rnd = SystemRandom()


class Lotteria:
    def __init__(self,
                 max_num=90,
                 max_sup=90,
                 length=6,
                 superstar=1):
        self.max_numbers = max_num
        self.max_superstar = max_sup
        self.length = length
        self.superstar = superstar

    @staticmethod
    def pallottoliere(num):
        while True:
            yield rnd.randint(1, num)

    @staticmethod
    def choice(length, num):
        numbers = list(range(1, num+1))

        def extraction():
            rnd.shuffle(numbers)
            single = rnd.choice(numbers)
            numbers.remove(single)
            
            return single
 
        return [extraction() for _ in range(length)]

    @staticmethod
    def sample(length, num):
        numbers = list(range(1, num+1))

        return rnd.sample(numbers, k=length)

    def int(self, length, num):
        combo = set()
        while len(combo) < length:
            combo.add(next(self.pallottoliere(num)))

        return combo

    @staticmethod
    def many(picker, length, num):
        samples = [picker(length, num) for _ in range(num)]
        rnd.shuffle(samples)
       
        return rnd.choice(samples)

    def __call__(self, backend='sample', many=True):
        
        if backend == 'choice':
            self.backend = 'choice'
            picker = self.choice

        if backend == 'sample':
            self.backend = 'sample'
            picker = self.sample

        if backend == 'int':
            self.backend = 'int'
            picker = self.int

        if not many:
            combo = picker(self.length, self.max_numbers) # type: ignore

            if self.superstar:
                super = picker(self.superstar, self.max_superstar) # type: ignore

        else:
            combo = self.many(picker, self.length, self.max_numbers) # type: ignore

            if self.superstar:
                super = self.many(picker, self.superstar, self.max_superstar) # type: ignore

        now = datetime.now().strftime("%d/%m/%Y %H:%M")

        print(f'Estrazione del: {now}', '\nnumbers combo:', *sorted(combo))
        if self.superstar:
            print('Superstar:', *sorted(super)) # type: ignore


In [2]:
superenalotto = Lotteria()


In [3]:
for b in ['choice', 'int', 'sample']:
    print('###', b, '###')
    superenalotto(backend=b, many=False)

print('\n################# many #########################\n')

for b in ['choice', 'int', 'sample']:
    print('###', b, '###')
    superenalotto(backend=b, many=True)

### choice ###
Estrazione del: 01/06/2023 12:34 
numbers combo: 6 14 24 28 63 83
Superstar: 53
### int ###
Estrazione del: 01/06/2023 12:34 
numbers combo: 1 11 32 40 61 67
Superstar: 67
### sample ###
Estrazione del: 01/06/2023 12:34 
numbers combo: 7 34 48 61 69 82
Superstar: 81

################# many #########################

### choice ###
Estrazione del: 01/06/2023 12:34 
numbers combo: 10 31 39 58 62 82
Superstar: 4
### int ###
Estrazione del: 01/06/2023 12:34 
numbers combo: 1 14 19 28 30 44
Superstar: 65
### sample ###
Estrazione del: 01/06/2023 12:34 
numbers combo: 20 43 56 61 63 68
Superstar: 51


In [4]:
eurojack = Lotteria(max_num=50, max_sup=12, length=5, superstar=2)


In [5]:
for b in ['choice', 'int', 'sample']:
    print('###', b, '###')
    eurojack(backend=b, many=False)

print('\n################# many #########################\n')

for b in ['choice', 'int', 'sample']:
    print('###', b, '###')
    eurojack(backend=b, many=True)

### choice ###
Estrazione del: 01/06/2023 12:34 
numbers combo: 13 15 22 26 39
Superstar: 2 9
### int ###
Estrazione del: 01/06/2023 12:34 
numbers combo: 5 7 16 29 45
Superstar: 9 11
### sample ###
Estrazione del: 01/06/2023 12:34 
numbers combo: 14 28 35 42 43
Superstar: 8 11

################# many #########################

### choice ###
Estrazione del: 01/06/2023 12:34 
numbers combo: 5 13 41 42 43
Superstar: 1 10
### int ###
Estrazione del: 01/06/2023 12:34 
numbers combo: 18 28 30 46 50
Superstar: 8 9
### sample ###
Estrazione del: 01/06/2023 12:34 
numbers combo: 5 13 28 35 43
Superstar: 3 9


In [6]:
superenalotto(backend='choice')

Estrazione del: 01/06/2023 12:34 
numbers combo: 6 14 18 64 79 89
Superstar: 51


In [7]:
eurojack()

Estrazione del: 01/06/2023 12:34 
numbers combo: 1 2 7 22 50
Superstar: 1 4
